# Objective: This script was created to see whether deepchem can give the same amount output that we get from _CORE_ for sklearn machine learning


<font
      color=red>Firstly, let's load in machine learning data and featurize it. If you load their built-in moleculenet datasets, deepchem automatically featurizes the molecules, normalize them and split them into train, test and validation. The featurization options are 'ECFP', 'GraphConv', 'Weave', 'Raw' (Convert SMILES to MOL object). There are also different data splitting method such as index, scaffold and random (which we are using).      </font>

In [1]:
from deepchem.molnet import load_lipo
lipo_task, lipo_datasets, transformers = load_lipo(split='random')
# 
train_dataset, valid_dataset, test_dataset = lipo_datasets  # lipo_datasets is a tuple with 3 values
print("Datatype:", type(train_dataset))

total_data = len(train_dataset) + len(valid_dataset) + len(test_dataset)
print("Dataset size:", total_data)
print("Number of training molecules:",len(train_dataset))
print("Training percentage:", len(train_dataset)/total_data)
print("Number of validating molecules:", len(valid_dataset))
print("Validating precentage:", len(valid_dataset)/total_data)
print("Number of testing molecules:", len(test_dataset))
print("Testing percentage:", len(test_dataset)/total_data)

C:\Users\quang\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\quang\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\quang\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a futur

Loading dataset from disk.
Loading dataset from disk.
Loading dataset from disk.
Datatype: <class 'deepchem.data.datasets.DiskDataset'>
Dataset size: 4200
Number of training molecules: 3360
Training percentage: 0.8
Number of validating molecules: 420
Validating precentage: 0.1
Number of testing molecules: 420
Testing percentage: 0.1


<font
      color=blue> While loading data from DeepChem is fast and somewhat convenient, the split data are class instances instead of a manipulaple datastructure such as dataframes, numpy arrays. While technically we can still add more featurization, now we have to convert each split into something that is easier to manipulate, convert SMILES into MOL object and add in RDKit features. The user also doesn't have the option to change the splitting ratio, which is set to 80/10/10. Since moleculenet was using ECFP as the main feature for their benchmarking, I assume this function was created to quicken the benchmarking process and not meant to be used in a normal machine learning pipeline.</font>
      
<font     
      color=red> Let's load in the data manually using Pandas instead. This will give us more flexibility on featurization and data splitting method.  </font>

In [2]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import PandasTools
from deepchem.feat import CircularFingerprint, RDKitDescriptors
import time

df = pd.read_csv("dataFiles/jak2_pic50.csv")
# PandasTools.AddMoleculeColumnToFrame(df, "smiles", 'mol', includeFingerprints=True)
mol = list(map(Chem.MolFromSmiles, df["SMILES"]))

t1 = time.perf_counter()
rdkit_desc = RDKitDescriptors().featurize(mol)  # Featurize molecules using built-in rdkit

#More about featurization in this turtorial: https://github.com/deepchem/deepchem/blob/master/examples/tutorials/06_Going_Deeper_on_Molecular_Featurizations.ipynb
t2 = time.perf_counter()
print(f"It took {t2-t1} secs to execute this method")

feature_list = list(RDKitDescriptors.allowedDescriptors)
print("Feature list", feature_list)
print("Number of features", len(feature_list))
feature_df = pd.DataFrame(rdkit_desc, columns=feature_list)
final_df = pd.concat([df, feature_df], axis=1)
# print(final_df)
# features = list(map(rdkit_desc, mol))

It took 28.967000999999982 secs to execute this method
Feature list ['EState_VSA5', 'NumSaturatedHeterocycles', 'Kappa3', 'Chi2n', 'NumAromaticCarbocycles', 'VSA_EState8', 'NumRadicalElectrons', 'MolWt', 'NHOHCount', 'PEOE_VSA5', 'VSA_EState1', 'NumHAcceptors', 'PEOE_VSA12', 'EState_VSA6', 'PEOE_VSA6', 'Chi3v', 'PEOE_VSA11', 'BalabanJ', 'PEOE_VSA14', 'SlogP_VSA5', 'SMR_VSA6', 'Chi2v', 'SlogP_VSA9', 'SMR_VSA2', 'HeavyAtomMolWt', 'SlogP_VSA3', 'RingCount', 'SlogP_VSA4', 'Ipc', 'MaxPartialCharge', 'VSA_EState4', 'Chi1v', 'SMR_VSA1', 'FractionCSP3', 'VSA_EState3', 'SMR_VSA4', 'VSA_EState5', 'VSA_EState7', 'SMR_VSA5', 'Chi3n', 'ExactMolWt', 'HeavyAtomCount', 'HallKierAlpha', 'TPSA', 'MinPartialCharge', 'SlogP_VSA8', 'PEOE_VSA9', 'Chi4n', 'NumSaturatedCarbocycles', 'NumHDonors', 'NumHeteroatoms', 'NumAromaticRings', 'MaxAbsEStateIndex', 'MinEStateIndex', 'SMR_VSA10', 'SlogP_VSA6', 'NOCount', 'EState_VSA1', 'NumAromaticHeterocycles', 'PEOE_VSA7', 'EState_VSA11', 'Chi0n', 'PEOE_VSA8', 'LabuteA

<font
      color=blue> While the steps are easy to remember for someone who's been doing this for a while, that is still a bit of work loading in data, turn SMILES into Mol-object, featurize it and combine it back to the original Dataframe. Before we move on to the final way of loading in data, let's look at the featurization process  .It took quite a while to calculate 111 features for the Lipo data. </font>
      
<font
      color=red> Using descriptastorus </font>

In [5]:
from descriptastorus.descriptors.DescriptorGenerator import MakeGenerator

generator = MakeGenerator(("RDKit2D",))
columns = []
for name, numpy_type in generator.GetColumns():
        columns.append(name)
t1 = time.perf_counter()
data = list(map(generator.process, df['SMILES']))
t2 = time.perf_counter()
print(f"It took {t2-t1} secs to execute this method")

print("Feature list", columns)
print("Number of features", len(columns))

It took 1.0404631999999765 secs to execute this method
Feature list ['RDKit2D_calculated', 'BalabanJ', 'BertzCT', 'Chi0', 'Chi0n', 'Chi0v', 'Chi1', 'Chi1n', 'Chi1v', 'Chi2n', 'Chi2v', 'Chi3n', 'Chi3v', 'Chi4n', 'Chi4v', 'EState_VSA1', 'EState_VSA10', 'EState_VSA11', 'EState_VSA2', 'EState_VSA3', 'EState_VSA4', 'EState_VSA5', 'EState_VSA6', 'EState_VSA7', 'EState_VSA8', 'EState_VSA9', 'ExactMolWt', 'FpDensityMorgan1', 'FpDensityMorgan2', 'FpDensityMorgan3', 'FractionCSP3', 'HallKierAlpha', 'HeavyAtomCount', 'HeavyAtomMolWt', 'Ipc', 'Kappa1', 'Kappa2', 'Kappa3', 'LabuteASA', 'MaxAbsEStateIndex', 'MaxAbsPartialCharge', 'MaxEStateIndex', 'MaxPartialCharge', 'MinAbsEStateIndex', 'MinAbsPartialCharge', 'MinEStateIndex', 'MinPartialCharge', 'MolLogP', 'MolMR', 'MolWt', 'NHOHCount', 'NOCount', 'NumAliphaticCarbocycles', 'NumAliphaticHeterocycles', 'NumAliphaticRings', 'NumAromaticCarbocycles', 'NumAromaticHeterocycles', 'NumAromaticRings', 'NumHAcceptors', 'NumHDonors', 'NumHeteroatoms', 'NumR

<font
      color=blue> Descriptastorus took a bit longer with 200 features. However, it provides much more feature methods and even use them in combinations. Whereas when we use DeepChem, we only have ECFP and a lesser version of descripstastorus' rdkit2d. However, since I am testing deepchem, I should keep using their product </font>
 
<font
      color=red> Back to loading data. Deepchem also provides another way to load machine learning data using the function _CSVLoader_. The user inputs in: target column name (tasks), SMILES column name (smiles_field) and the molecular feature they want to use. </font>

In [ ]:
import deepchem as dc
featurizer = RDKitDescriptors()  # rdkit2d(ish) features
 
loader = dc.data.CSVLoader(
        tasks =["pIC50"], smiles_field="SMILES",
        featurizer=featurizer
                    )
dataset = loader.featurize("dataFiles/jak2_pic50.csv")
print("Final variable type", type(dataset))

<font
      color=blue> This method is the best so far. Since I'm importing a foreign CSV from my folder instead of DeepChem's, it is safe to say that _CSVLoader_ is built for general usage. However, I'm getting a DeepChem class object as the final product instead of a datastructure. This means that the "dataset" variable can only work with DeepChem function and nothing else. This is because they are using __yield__ instead of __return__. I wonder why  </font>
      
<font
      color=green> According to the documentation, DeepChem loads in data in chunk, which are called _shards_ , instead of everything at once for memory purposes. This created a for loop in the coding process, which is why they have to use __yield__ , which returns a class object instead of a data structure. (Read __load_csv_file__ in https://deepchem.io/docs/_modules/deepchem/utils/save.html#load_csv_files and ___get_shards__ in  https://github.com/deepchem/deepchem/blob/614d35913abbef35401fb5421ab5937fdc9fcf6b/deepchem/data/data_loader.py </font>

<font
      color=red> Let's move on to the data splitting process now that we have featurized our data. I'm splitting the data into train/test/validation instead of just train/test. This is because I want to see what the validation process looks like in DeepChem </font>

In [ ]:
splitter = dc.splits.RandomSplitter()
train_dataset, valid_dataset, test_dataset = splitter.train_valid_test_split(dataset) 

total_data = len(train_dataset) + len(valid_dataset) + len(test_dataset)
print("Dataset size:", total_data)
print("Number of training molecules:",len(train_dataset))
print("Training percentage:", len(train_dataset)/total_data)
print("Number of validating molecules:", len(valid_dataset))
print("Validating precentage:", len(valid_dataset)/total_data)
print("Number of testing molecules:", len(test_dataset))
print("Testing percentage:", len(test_dataset)/total_data)

<font
      color=red> We finally discover a major difference between using DeepChem and using _core_ - or even using sklearn in general. Usually, when we use sklearn's train_test_split, each split is divided into a feature group and a target group. Meaning we would each have a feature set and a target set for training data, test data and validation data. However, when I load in the data using DeepChem's CSVLoader, the code will turn the target, features, SMILES and even weight vector into "class methods" of the variable. That's why you don't see different data group separated or even the removal of the SMILES column before splitting.   </font>

<font
      color=green>  Read https://deepchem.io/docs/_modules/deepchem/data/datasets.html#DiskDataset for more detail  </font>

<font 
      color=blue>  Let me demonstrate</font>

In [ ]:
print("SMILES from training set:",  train_dataset.ids)
print("Training SMILES length", len(train_dataset.ids))
print("Training set features:", train_dataset.X)
print("Training features length:", len(train_dataset.X))
print("Training features data type:", type(train_dataset.X))
print("Training target:", train_dataset.y)
print("Training target length:", len(train_dataset.y))

<font
      color=red> As you can see, SMILES, target data and features are all methods of the variable __train_dataset__ . I really like this internal feature. It reduces the amount of variable the user has to account for after splittting the data. If we were using sklearn _train_test_split_ , we would have to use 6 different variables instead of 3.  </font>
      
<font
      color=blue> Let's move on to the data normalization process. Unfortunately, if you're doing sklearn machine learning though DeepChem, you are forced to normalize your data. Or else, you don't have enough variables for the actual machine learning process. Furthermore, they only offer one normalization method for sklearn machine learning. However, it is interesting how you can choose to normalize features data and target data. I'll just normalize the features for now. </font>

In [ ]:
transformers = [
    dc.trans.NormalizationTransformer(transform_y=True, dataset=dataset)]

for dataset in [train_dataset, valid_dataset, test_dataset]:
    for transformer in transformers:
        dataset = transformer.transform(dataset)

In [ ]:
from sklearn.svm import SVR

sklearn_model = SVR()
model = dc.models.SklearnModel(sklearn_model)
model.fit(train_dataset)


<font
      color=red> Nothing new here except that we only need one variable for the __fit()__ instead of 2.  </font>
 
 <font
      color=blue> We next evaluate the model on the validation data to evaluate SVR predictive power without hyperparameter tuning. One more plus is that you can compute multiple metrics at a time. I'm using RMSE, R2 and AUC as the metrics for this process.  </font>

In [ ]:
from deepchem.utils.evaluate import Evaluator

metrics = [dc.metrics.Metric(dc.metrics.rms_score), dc.metrics.Metric(dc.metrics.r2_score), dc.metrics.Metric(dc.metrics.auc, mode="regression")]  # Define what metrics I want to use
evaluator = Evaluator(model, valid_dataset, transformers)  # Load in required data before the evaluation process
score = evaluator.compute_model_performance(metrics)  #Compute the score using the provided metric(s).
print(score)

<font
      color=red> The scores wasn't as good as I expected. Oh well.</font>

 <font
      color=blue> Next step in the process is the most exciting one, Hyperparameter tuning!   </font>

In [ ]:
""" The only way I was able to use the HyperparamOpt function was to create this module builder function. In every 
DeepChem's tutorials that contain hyperparameter tuning, they all follow this format
"""

def model_builder(model_params, model_dir):
  sklearn_model = SVR(**model_params)
  return dc.models.SklearnModel(sklearn_model, model_dir)



params_dict = {
    "C" : [0.001, 0.005, 0.01, 10, 100],
    "epsilon" : [0.1,0.2,0.3,0.4, 0.5,0.6],
    "gamma" : [0.001, 0.01, 1]
}

metric = dc.metrics.Metric(dc.metrics.rms_score)
optimizer = dc.hyper.HyperparamOpt(model_builder)
best_model, best_hyperparams, all_results = optimizer.hyperparam_search(
    params_dict, train_dataset, valid_dataset, transformers,
    metric=metric)


<font
      color=red> It seems like DeepChem's HyperparamOpt function is basically sklearn's grid search. Which is exhaustive but very time consuming. DeepChem only provides callbacks options for keras NN. However, I want to try out their GaussianProcessHyperparamOpt, a Bayesian optimization, and see if their is any major difference.  </font>

<font
      color=green> The package they use for Bayesian: https://pygpgo.readthedocs.io/en/latest/</font>

<font
      color=blue> Next step is to use test set to evaluate the model's performance  </font>

In [ ]:
test_evaluator = Evaluator(best_model, test_dataset, transformers)
test_score = test_evaluator.compute_model_performance([metric])
print(test_score)

<font
      color=blue> The next course of action is to plot the predicted $RMSE$ scores versus the true $RMSE$ scores for the constructed model. However, I could not find anything in DeepChem that resembles a general graphing function. In almost all of their tutorial, this is where the code ends, without graphing anything. And when they do graph, the entire process was done manually though $matplotlib$ </font>

In [ ]:
%matplotlib inline
import matplotlib
from matplotlib import cm
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score, classification_report, confusion_matrix, accuracy_score, roc_auc_score

predicted_test = best_model.predict(test_dataset)
save = best_model.save
true_test = test_dataset.y
r2 = r2_score(true_test, predicted_test)
print(r2)
mse = mean_squared_error(true_test, predicted_test)
print(mse)
rmse = np.sqrt(mean_squared_error(true_test, predicted_test))
plt.rcParams['figure.figsize'] = [12, 9]
plt.style.use('bmh')
fig, ax = plt.subplots()

x = true_test
y = predicted_test

plt.scatter(x, y, alpha=0.7)
lims = [np.min([ax.get_xlim(), ax.get_ylim()]),
            np.max([ax.get_xlim(), ax.get_ylim()])
            ]
    # set axis limits


plt.xlabel('True', fontsize=14)
plt.ylabel('Predicted', fontsize=14)

plt.plot(lims, lims, 'k-', label='y=x')
plt.plot([], [], ' ', label='R^2 = %.3f' % r2)
plt.plot([], [], ' ', label='RMSE = %.3f' % rmse)
plt.plot([], [], ' ', label='MSE = %.3f' % mse)
ax.set_aspect('equal')
ax.set_xlim(lims)
ax.set_ylim(lims)
    # plt.axis([-2,5,-2,5]) #[-2,5,-2,5]
ax.legend(prop={'size': 16}, facecolor='w', edgecolor='k', shadow=True)

fig.patch.set_facecolor('blue')  # Will change background color
fig.patch.set_alpha(0.0)  # Makes background transparent

<font
      color=red> The result reminds me of the Quantum data. I wonder if I did something wrong. I've never run ML using jak2 dataset before so I don't know what the result should look like. Assuming that I did not mess up, I think that DeepChem's normalization process is causing the problem</font>

<font
      color=blue> Anyway, let's move on. I would like to use GaussianProcessHyperparamOpt instead of HyperparamOpt, which is basically grid search. Since it is also a Bayesian optimization, my hope is to get a better result at a faster time.</font>

In [ ]:
import numpy as np

def model_builder(model_params, model_dir):
  sklearn_model = SVR(**model_params)
  return dc.models.SklearnModel(sklearn_model, model_dir)

params_dict = {
    "C" : [0.001, 0.005, 0.01, 10, 100],
    "epsilon" : [0.1,0.2,0.3,0.4, 0.5,0.6],
    "gamma" : [0.001, 0.01, 1]
}

metric = [
            dc.metrics.Metric(dc.metrics.pearson_r2_score)
        ]
optimizer = dc.hyper.GaussianProcessHyperparamOpt(model_builder)
hyper_parameters, valid_performance_opt = optimizer.hyperparam_search(
    params_dict, train_dataset, valid_dataset, transformers,
    metric=metric)

<font
      color=red> I've been running to some issues regarding GaussianProcessHyperparamOpt which are reported in DeepChem's Issue page on Github. Guess this is as far as I can go for now. </font>

# Comments

## Things that _core_ and _DeepChem_ have in common in terms of sklearn machine learning

- A way to rapidly load in supported data and featurize them
- Include Rdkit's Morgan fingerprint and its 2d features
- Data normalization
- Support almost all, if not all sklearn machine learning algorithm
- Hyperparameter tuning.
- A way to compute all statistical metrics such as R2, RMSE, AUC, ...
- A way to save and reload sklearn model using __joblib__
## Things that make _core_ better

- Many more rdkit featurization. You can even combine them with each other.
- Much more flexiblity for the user in terms of choices. I felt forced to follow exactly what DeepChem wants for their ML process.
- _core_ has a working Bayesian optimizer. DeepChem uses grid search
- We made graphing functions for the analytical process. The user has to manually build the graph using DeepChem
- Our analytical process is much better since we have variable importance for supported algorithms and uncertainty graph. DeepChem only has an _prediction uncertainty_ function for Deeplearning algorithms 
- Overall, there is still a lot of coding involve when using DeepChem. While there are functions for almost everything, the user still has to lay everything out step by step: loading data, splitting data, fitting, ... etc, to get the final result. In _core_, we only run "command lines" to input in our options for different steps in the process: what dataset, what algorithm, ...

## Things that make DeepChem better

- Their _CSVLoader_ function. It's really nice how parts of the dataset are turned into methods so that everything is stored in one variable. Very nifty
- Most of their data inputing process such as loading CSV file or featurizations are done in chunks. I don't know if this is "better" since it makes things a bit slower but I think it will be helpful when dealing with large and complex datasets or featurizations.
- Even though I did not test this, they also have incredible featurization for other powerful deeplearning algorithm such as __graph convolutional__ and __weave__.
- They have wonderful documentations and tutorials. While it did took a while for me to find an explanation foe everything, they have lots of comments and explanations throughout their code that I rarely see in other modules.
- They made the benchmarking process between different algorithms, features and dataset very fast and convenient. 
## Conclusion
- Overall, in terms of sklearn machine learning, _core_ is easier, more flexible and more convenient to use than DeepChem.
- However, they focus much more effort into deeplearning and I think sklearn's algorithms were included because of their popularity and DeepChem can compare them against other complex neural networks. 

## Next plan
- Next, I will be diving into their deeplearning process. I think will be much more interesting since this is what made DeepChem so well-known. 